In [1]:
import numpy as np
import pandas as pd
from scipy.ndimage import uniform_filter
from scipy.stats import entropy
import cv2
import os

In [2]:
def extract_rgb_features(image_path):
    """Computes RGB color statistics for a given image."""
    
    image = cv2.imread(image_path)
    image = image.astype(np.float32)

    # Split into RGB channels
    B, G, R = cv2.split(image)

    # Create a mask to ignore black pixels (where all channels are 0)
    maskR = (R > 0)
    maskG = (G > 0)
    maskB = (B > 0)

    # Compute mean for each channel
    mean_R = np.mean(R[maskR]) if np.any(maskR) else 0
    mean_G = np.mean(G[maskG]) if np.any(maskG) else 0
    mean_B = np.mean(B[maskB]) if np.any(maskB) else 0

    # Compute Mean (R - G) ignoring black pixels
    maskR_G = (R>0) | (G>0)
    mean_R_G = np.mean((R - G)[maskR_G]) if np.any(maskR_G) else 0

    return {
        "Mean_Red": mean_R,
        "Mean_Green": mean_G,
        "Mean_Blue": mean_B,
        "Mean_R-G": mean_R_G
    }

In [3]:
def extract_erythema_index(image_path):
    """Computes the Erythema Index (EI) for a given image."""
    
    image = cv2.imread(image_path)
    b, g, r = cv2.split(image)
    r, g = r.astype(np.int32), g.astype(np.int32)

    # Create a mask to ignore completely black pixels (where R and G are both 0)
    mask = (r > 0) | (g > 0)

    # Initialize EI array with zeros
    ei_values = np.zeros_like(r, dtype=np.float32)

    # Avoid division by zero
    valid_pixels = (r + g) > 0  
    ei_values[valid_pixels] = 100 * ((r - g)[valid_pixels] / (r + g)[valid_pixels])

    # Compute the mean EI for the entire image (excluding black pixels)
    mean_ei = np.mean(ei_values[mask]) if np.any(mask) else 0

    return {"Erythema_Index": mean_ei}


In [4]:
def extract_hsv_features(image_path):
    """Computes mean Hue, Saturation, and Value (HSV) features for a given image."""
    
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    h, s, v = h.astype(np.int32), s.astype(np.int32), v.astype(np.int32)

    # Create a mask to ignore black pixels (where V == 0)
    mask = v > 0

    # Compute mean values only for valid (non-black) pixels
    mean_h = np.mean(h[mask]) if np.any(mask) else 0
    mean_s = np.mean(s[mask]) if np.any(mask) else 0
    mean_v = np.mean(v[mask]) if np.any(mask) else 0

    return {
        "Mean_Hue": mean_h,
        "Mean_Saturation": mean_s,
        "Mean_Value": mean_v
    }

In [5]:
def extract_hhr(image_path, H_thresh=100):
    """Computes High Hue Ratio (HHR) for a given image based on a hue threshold."""

    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    h, v = h.astype(np.int32), v.astype(np.int32)

    # Create a mask to exclude black pixels (V == 0)
    valid_pixels_mask = v > 0  

    # Mask high hue pixels (H > H_thresh)
    high_hue_mask = (h > H_thresh) & valid_pixels_mask  

    # Compute High Hue Ratio (HHR)
    high_hue_count = np.count_nonzero(high_hue_mask)
    valid_pixel_count = np.count_nonzero(valid_pixels_mask)

    HHR = (high_hue_count / valid_pixel_count) if valid_pixel_count > 0 else 0

    return {"High_Hue_Ratio": HHR}


In [6]:
def extract_brightness(image_path):
    """Computes the mean brightness (B) of a given image, excluding black pixels."""
    
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a mask to ignore completely black pixels (where gray == 0)
    valid_pixels_mask = gray > 0  

    # Compute mean brightness, excluding black pixels
    mean_brightness = np.mean(gray[valid_pixels_mask]) if np.any(valid_pixels_mask) else 0

    return {"Brightness": mean_brightness}


In [7]:
def extract_entropy(image_path):
    """Computes the entropy (Ent) of a given image using pixel intensity distribution."""
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Compute histogram (256 bins for intensity levels 0-255)
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])

    # Normalize histogram to get probability distribution
    hist_norm = hist.ravel() / hist.sum()  # Convert to 1D array and normalize

    # Compute entropy
    image_entropy = entropy(hist_norm, base=2)

    return {"Entropy": image_entropy}

In [8]:
def extract_gray_level_features(image_path, window_size=9):
    """Computes gray-level features (G1-G5) for a given image using local window analysis."""
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Create mask: Ignore completely black pixels (intensity == 0)
    mask = image > 0 

    # Convert image to float32 before processing to prevent integer NaN errors
    image_float = image.astype(np.float32)

    # G1: Difference between pixel intensity and min intensity in window
    G1 = image_float - cv2.erode(image_float, np.ones((window_size, window_size), np.uint8))

    # G2: Difference between max intensity in window and pixel intensity
    G2 = cv2.dilate(image_float, np.ones((window_size, window_size), np.uint8)) - image_float

    # G3: Difference between pixel intensity and mean intensity in window
    mean_window = uniform_filter(image_float, size=window_size)
    G3 = image_float - mean_window

    # G4: Standard deviation in the window
    def local_std(img):
        mean_sq = uniform_filter(img ** 2, size=window_size)
        return np.sqrt(mean_sq - mean_window**2)

    G4 = local_std(image_float)

    # G5: Center pixel intensity (original grayscale image)
    G5 = image_float.copy()

    # Apply mask: Set black pixels to NaN to exclude from analysis
    G1[~mask] = np.nan
    G2[~mask] = np.nan
    G3[~mask] = np.nan
    G4[~mask] = np.nan
    G5[~mask] = np.nan

    # Compute mean values **only for valid pixels (non-black)**
    mean_G1 = np.nanmean(G1)
    mean_G2 = np.nanmean(G2)
    mean_G3 = np.nanmean(G3)
    mean_G4 = np.nanmean(G4)
    mean_G5 = np.nanmean(G5)

    return {
        "Mean_G1": mean_G1,
        "Mean_G2": mean_G2,
        "Mean_G3": mean_G3,
        "Mean_G4": mean_G4,
        "Mean_G5": mean_G5
    }

In [9]:
def extract_lab_features(image_path):
    """Computes mean L (Lightness), A (Green-Red), and B (Blue-Yellow) features for a given image."""
    
    image = cv2.imread(image_path)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)
    L, A, B = L.astype(np.float32), A.astype(np.float32), B.astype(np.float32)

    # Create a mask to exclude black pixels (where L == 0)
    mask = L > 0

    # Compute mean values only for valid (non-black) pixels
    mean_L = np.mean(L[mask]) if np.any(mask) else 0
    mean_A = np.mean(A[mask]) if np.any(mask) else 0
    mean_B = np.mean(B[mask]) if np.any(mask) else 0

    return {
        "Mean_L": mean_L,
        "Mean_A": mean_A,
        "Mean_B": mean_B
    }

In [10]:
image_folder = "../Produced Images/Fingernail Masked"

In [11]:
data = []
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]

for image_name in image_files:
    image = os.path.join(image_folder, image_name)
   

    # Extract features
    features = {"Image_Name": image_name}
    features.update(extract_rgb_features(image))
    features.update(extract_erythema_index(image))
    features.update(extract_hsv_features(image))
    features.update(extract_hhr(image))
    features.update(extract_brightness(image))
    features.update(extract_entropy(image))
    features.update(extract_lab_features(image))
    features.update(extract_gray_level_features(image))

    # Append to data list
    data.append(features)

# Create DataFrame
df = pd.DataFrame(data)

In [12]:
df.head()

,Image_Name,Mean_Red,Mean_Green,Mean_Blue,Mean_R-G,Erythema_Index,Mean_Hue,Mean_Saturation,Mean_Value,High_Hue_Ratio,Brightness,Entropy,Mean_L,Mean_A,Mean_B,Mean_G1,Mean_G2,Mean_G3,Mean_G4,Mean_G5
0,1710068253661.jpg,134.050552,96.584320,99.731155,39.987572,20.910883,81.653090,121.234397,122.423960,0.450395,101.867837,0.309940,110.529648,145.197021,134.773117,27.924637,19.133512,2.807380,13.223565,115.026421
1,1710068488123.jpg,130.116638,96.251602,108.679932,35.943634,19.991240,149.553003,106.154819,122.538280,0.857677,103.105610,0.581566,110.493759,144.768967,129.353775,21.176699,15.277468,1.794092,10.468807,112.591042
2,1710068973167.jpg,129.355988,102.929359,112.637650,30.646730,18.173225,145.661875,101.860296,120.641513,0.829408,106.407896,0.328764,115.054176,142.222137,129.317017,30.393871,19.295452,3.140271,14.342509,117.918045
3,1710069174769.jpg,143.493515,107.775490,115.031937,39.136528,19.121464,127.990956,102.576986,134.948959,0.720499,114.226762,0.421715,122.604485,145.074997,131.939560,27.475504,18.762531,3.331137,13.420000,124.867531
4,1710069438045.jpg,132.607178,91.582443,99.490021,43.501892,22.581299,124.312524,117.857348,123.905760,0.701134,99.732496,0.432294,108.327377,147.631821,132.509552,22.289457,13.314292,2.737524,10.745696,110.416039


In [13]:
df.shape

(425, 20)

In [14]:
df1 = pd.read_excel("../Dataset Used/Blood Report_without patient.xlsx",sheet_name="right_nail")
df1.head()

,2. Take a photo of right hand fingernail,Blood Sample Id,Unnamed: 2,Unnamed: 3,Annotated_Data,Avaiable ID,Hb Value
0,1709617374918.jpg,202403051,NaN,NaN,1709617374918.jpg,202403051.0,11.5
1,1709619498294.jpg,202403058,NaN,NaN,1709617703730.jpg,202403052.0,11.6
2,1709621336576.jpg,2024030512,NaN,NaN,1709618658975.jpg,202403056.0,11.1
3,1709624176391.jpg,2024030518,NaN,NaN,1709619498294.jpg,202403058.0,12.1
4,1709625695477.jpg,2024030522,NaN,NaN,1709620017738.jpg,202403059.0,12.2


In [15]:
df1_filtered = df1[['Annotated_Data','Hb Value']].rename(columns={'Annotated_Data':'Image_Name'})
df1_filtered.head()

,Image_Name,Hb Value
0,1709617374918.jpg,11.5
1,1709617703730.jpg,11.6
2,1709618658975.jpg,11.1
3,1709619498294.jpg,12.1
4,1709620017738.jpg,12.2


In [16]:
df2 = pd.read_excel("../Dataset Used/Updated Healthworker Form Part 1_without patient identifier.xlsx") 
df2.head()

,start,end,today,deviceid,username,Do you give consent?,Patient Id,Blood Sample Id,2. Enter age in completed years,4. Patient Sex,...,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,2024-03-05T11:01:43.141+05:30,2024-03-05T12:36:06.048+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,1,202403051,63,Female,...,116,e7525154-4a03-4818-b76a-85f5d5657f73,2024-03-05T17:57:45,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,1
1,2024-03-05T11:39:13.925+05:30,2024-03-05T11:55:12.666+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,8,202403058,42,Female,...,117,a407f9d7-8643-4a35-a614-3b83184b83e0,2024-03-05T17:57:46,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,2
2,2024-03-05T12:07:50.954+05:30,2024-03-05T12:26:37.006+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,12,2024030512,65,Male,...,119,3f1f095e-d9d1-4a6c-bf3e-616632cf64ac,2024-03-05T17:57:47,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,3
3,2024-03-05T12:50:11.280+05:30,2024-03-05T13:13:53.977+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,18,2024030518,75,Female,...,121,7be32b46-35d2-4923-b256-ef11b633fd05,2024-03-05T17:57:52,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,4
4,2024-03-05T13:21:52.005+05:30,2024-03-05T13:40:16.946+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,22,2024030522,38,Female,...,122,b7d5c35e-cba8-46ff-9e2e-9d82ec8eedc5,2024-03-05T17:57:54,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,5


In [23]:
df2_filtered = df2[['2. Take a photo of right hand fingernail','4. Patient Sex', '2. Enter age in completed years','8. Height (in cm)','9. Weight (in kg)']].rename(
    columns={'4. Patient Sex': 'Gender', '2. Enter age in completed years': 'Age','2. Take a photo of right hand fingernail':'Image_Name','8. Height (in cm)':'Height','9. Weight (in kg)':'Weight'}
)

In [24]:
df2_filtered['Gender'] = df2_filtered['Gender'].map({'Male':0, 'Female':1})
df2_filtered.head()

,Image_Name,Gender,Age,Height,Weight
0,1709617374918.jpg,1,63,150.0,42.0
1,1709619498294.jpg,1,42,155.0,45.0
2,1709621336576.jpg,0,65,158.0,60.0
3,1709624176391.jpg,1,75,149.0,34.0
4,1709625695477.jpg,1,38,145.0,38.0


In [25]:
df1_filtered.shape

(426, 2)

In [26]:
df2_filtered.shape

(426, 5)

In [27]:
merge_df = pd.merge(df1_filtered,df2_filtered,on='Image_Name',how='inner')
merge_df.head()

,Image_Name,Hb Value,Gender,Age,Height,Weight
0,1709617374918.jpg,11.5,1,63,150.0,42.0
1,1709617703730.jpg,11.6,0,63,155.0,55.0
2,1709618658975.jpg,11.1,1,50,151.0,45.0
3,1709619498294.jpg,12.1,1,42,155.0,45.0
4,1709620017738.jpg,12.2,1,76,148.0,60.0


In [28]:
merge_df.shape

(417, 6)

In [29]:
final_df = pd.merge(merge_df,df,on='Image_Name',how='inner')

In [30]:
final_df.head()

,Image_Name,Hb Value,Gender,Age,Height,Weight,Mean_Red,Mean_Green,Mean_Blue,Mean_R-G,...,Brightness,Entropy,Mean_L,Mean_A,Mean_B,Mean_G1,Mean_G2,Mean_G3,Mean_G4,Mean_G5
0,1709617374918.jpg,11.5,1,63,150.0,42.0,138.606735,92.703957,82.860825,48.400242,...,94.860514,0.161835,106.644852,148.352493,142.836365,41.225063,24.367199,4.813530,19.873451,118.198151
1,1709617703730.jpg,11.6,0,63,155.0,55.0,163.522308,133.304199,128.327576,35.787098,...,133.348545,0.384943,143.963776,141.686874,136.985916,33.229301,20.028419,3.434504,15.541560,150.014755
2,1709618658975.jpg,11.1,1,50,151.0,45.0,128.885925,86.655060,83.011177,43.607315,...,91.597666,0.255604,101.135132,146.415466,139.223053,28.985512,19.525328,2.999077,13.878590,106.227722
3,1709619498294.jpg,12.1,1,42,155.0,45.0,147.055420,94.376381,83.437386,54.422375,...,101.526466,0.363412,112.458321,150.253754,144.512558,25.358034,15.315536,2.720641,11.993537,117.244637
4,1709620017738.jpg,12.2,1,76,148.0,60.0,147.698196,108.049637,104.699692,42.440662,...,112.915589,0.392463,122.273796,145.091568,137.814453,26.433331,17.215773,2.404404,12.501428,126.604927


In [31]:
final_df.to_excel('FingerNail_Features.xlsx',index=False)